In [ ]:
# Now we will leverage the fine-tuned cross-encoder (03_CrossEncoder_tuning) to score the augmented activity label pairs 

In [19]:
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util, LoggingHandler, SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
from zipfile import ZipFile
import logging
import csv
import sys
import torch
import math
import gzip
import os
import numpy as np

In [101]:
source_path = '' # specify source path of file with pairs that should be scored
output_path = '' # specify output path for scored pairs

In [ ]:
##################################################################
#
# Score activity label pairs using the previously trained cross-encoder (BERT) model
#
##################################################################

cross_encoder_path = '' #specify path to the fine-tuned cross-encoder

#Loading the fine-tuned cross-encoder
cross_encoder = CrossEncoder(cross_encoder_path)

silver_data = []
       
with open(os.path.join(source_path), encoding='utf8') as fIn: 
    reader = csv.DictReader(fIn, delimiter=';', quoting=csv.QUOTE_NONE)
    for row in reader: 
        silver_data.append([row['label1'], row['label2']])

silver_scores = cross_encoder.predict(silver_data)

# All model predictions should be between [0,1]
assert all(0.0 <= score <= 1.0 for score in silver_scores)

binary_silver_scores = [1 if score >= 0.7 else 0 for score in silver_scores] # If the retrieved scores should additionally be saved as binary classes, the thrershold needs to specified here.

In [162]:
labels_scores_matched = list(zip(silver_data, silver_scores, binary_silver_scores))

In [163]:
import pandas as pd
df = pd.DataFrame(labels_scores_matched, columns = ['pair','score','annotation']) 
#df.to_csv('labels_matched.csv')

In [164]:
# Saving dataframe without duplicates as .csv file

header = ["pair", "score", "annotation"]
#df.to_csv('output.csv', columns = header)

df.to_csv(output_path, sep=',', columns = header, header=True, index=False)# sep=';')